In [2]:
!pip install pymupdf easyocr opencv-python pillow # easy ocr
!pip install pymupdf4llm
!pip install pyhwp beautifulsoup4
!pip install langchain_community langchain_google_genai faiss-cpu

In [3]:
# 필수 빌드 도구 & 라이브러리 설치
!apt-get update
!apt-get install -y build-essential libgsf-1-dev libxml2-dev libglib2.0-dev libiconv-hook-dev

# 소스코드 clone
!git clone https://github.com/mete0r/pyhwp.git
%cd pyhwp

# 설치
!python setup.py install

%cd ..

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [80.4 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]           
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,794 kB]
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]     
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,374 kB]


In [18]:
import io
import os
import re
import cv2
import time
import fitz  # PyMuPDF
import pyhwp
import gdown
import torch
import easyocr
import zipfile
import pickle
import easyocr
import subprocess
import pymupdf4llm
import pandas as pd
import numpy as np
from PIL import Image
from datetime import datetime
from bs4 import BeautifulSoup, NavigableString, Comment

from langchain.schema import Document

## 기본 Utils (basic_utils.py)

In [16]:
def load_secret(key_name: str):

    # 1) Kaggle 환경: os.environ 또는 파일에서 읽기
    if key_name in os.environ:
        return os.environ[key_name]

    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        return user_secrets.get_secret(key_name)
    except ImportError:
        pass

    # 2) Colab 환경인지 확인
    try:
        import google.colab.userdata as userdata
        try:
            return userdata.get(key_name)
        except KeyError:
            pass # Key not in Colab userdata
    except ImportError:
        pass

    raise KeyError(f"Secret '{key_name}' not found in Colab userdata, os.environ, or file.")


def download(id, filename):
    if os.path.isfile(filename) and os.path.getsize(filename) > 0:
        print(f"[skip] {filename} 이미 존재합니다.")
    else:
        gdown.download(id=id, output=filename, quiet=False)
        print(f'[ok] {filename} 다운로드 완료.')

        if filename.lower().endswith('.zip'):
            try:
                with zipfile.ZipFile(filename, 'r') as zip_ref:
                    zip_ref.extractall(os.path.dirname(filename) or '.') # Extract to directory of the file, or current if no directory
                print(f'[ok] {filename} 압축풀기 완료.')
            except zipfile.BadZipFile:
                print(f"[warning] {filename}은 유효한 zip 파일이 아닙니다.")

    return filename


def load_csv(filename):
  """Loads a CSV file into a pandas DataFrame."""
  try:
    df = pd.read_csv(filename)
    print(f"[ok] {filename} 로드 완료.")
    return df
  except FileNotFoundError:
    print(f"[error] {filename} 파일을 찾을 수 없습니다.")
    return None
  except Exception as e:
    print(f"[error] {filename} 로드 중 오류 발생: {e}")
    return None


def save_pages(pages, filename):
    directory = os.path.dirname(filename)
    if directory and not os.path.exists(directory):
        os.makedirs(directory)
        print(f"디렉토리 '{directory}'를 생성했습니다.")
    with open(filename, 'wb') as f:
        pickle.dump(pages, f)
    print(f"'{filename}' 파일에 pages를 저장했습니다.")


def load_pages(filename):
    if not os.path.exists(filename):
        print(f"오류: '{filename}' 파일을 찾을 수 없습니다.")
        return None
    with open(filename, 'rb') as f:
        pages = pickle.load(f)
    print(f"'{filename}' 파일에서 pages를 로드했습니다.")
    return pages


def timer(func):
    start_time = time.time()
    result = func()
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"실행 시간: {elapsed_time:.4f}초")
    return result

def ext(original_filename, ext='pkl'):
  base_filename, _ = os.path.splitext(original_filename)
  return f"{base_filename}.{ext}"


def firstname(original_filename):
    filename_with_extension = os.path.basename(original_filename)
    base_filename, _ = os.path.splitext(filename_with_extension)
    return base_filename

## PDF/HWP 문서 Utils (document_utils.py)

In [6]:
def load_pymupdf(pdf_path, filename, langs=['ko','en'], zoom=3.0, gpu=True):
    """
    PDF 페이지별 텍스트 레이어 + EasyOCR 결과 병합
    - langs: EasyOCR 언어 리스트
    - zoom: 해상도 배율 (3.0이면 약 216dpi)
    - gpu: GPU 사용 여부
    """
    # 캐시 로드
    output_path = f'outputs/{filename}'
    if os.path.exists(output_path):
        return load_pages(output_path)

    # EasyOCR Reader 초기화
    reader = easyocr.Reader(langs, gpu=gpu)

    doc = fitz.open(f'files/{pdf_path}')
    pages = []

    for page_num, page in enumerate(doc, start=1):
        # 1. 텍스트 레이어 추출
        text_layer = page.get_text()

        # 2. 고해상도 렌더링
        matrix = fitz.Matrix(zoom, zoom)
        pix = page.get_pixmap(matrix=matrix, alpha=False)
        img = Image.open(io.BytesIO(pix.tobytes("png")))

        # 3. EasyOCR 실행
        cv_img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        results = reader.readtext(cv_img, detail=0)  # detail=0 → 텍스트만 리스트로 반환
        ocr_text = "\n".join(results)

        # 4. 병합
        merged_text = (text_layer or "").strip() + "\n" + (ocr_text or "").strip()

        print(f"[페이지 {page_num}] 텍스트 길이: {len(text_layer)}, OCR 길이: {len(ocr_text)}")
        pages.append(Document(page_content=merged_text))
        display(img)
        print(ocr_text)
    doc.close()

    # 캐시 저장
    save_pages(pages, output_path)
    return pages


def load_pymupdf4llm_easyocr(pdf_filename, metadata = {}):
    IMAGE_PATH = 'images'
    pdf_path = f'files/{pdf_filename}'
    output_path = f'outputs/{ext(pdf_filename)}'
    if os.path.exists(output_path):
        return load_pages(output_path)
    
    os.makedirs(IMAGE_PATH, exist_ok=True)
    
    # EasyOCR 리더 객체 초기화 (GPU 사용 가능 시 활용)
    reader = easyocr.Reader(['ko', 'en'], gpu=torch.cuda.is_available())

    # 1. PyMuPDF4LLM을 사용해 마크다운 생성 및 이미지 추출
    # write_images=True 옵션으로 이미지 추출
    # image_path 옵션으로 이미지 저장 위치 지정
    # page_chunks=True 옵션으로 페이지별로 분리된 결과 생성
    extracted_chunks = pymupdf4llm.to_markdown(pdf_path, write_images=True, image_path=IMAGE_PATH, page_chunks=True)

    # final_markdown = ""
    pages = []
    # 2. EasyOCR로 이미지 내 텍스트 인식 후 마크다운에 추가
    for chunk in extracted_chunks:
        page_text = chunk["text"]
        
        page_number = chunk["metadata"]["page"] # Get the page index from metadata
        print('페이지: ', page_number)
        
        # 마크다운 내 이미지 참조 찾기 (예: ![image_name](images/image_name.png))
        # ([^\]]+)는 이미지 이름 부분을 캡처
        image_ref_pattern = re.compile(r"!\[(.*?)\]\((.*?)\)")
        matches = image_ref_pattern.finditer(page_text)
        
        # 이미지 참조를 순서대로 처리
        for match in reversed(list(matches)):
            image_path = match.group(2)
            
            # EasyOCR로 이미지 내 텍스트 인식
            try:
                ocr_results = reader.readtext(image_path)
                ocr_caption = " ".join([text for _, text, _ in ocr_results])
            except Exception as e:
                print(f"OCR 처리 중 오류 발생: {e}")
                ocr_caption = "OCR 처리 실패"
            
            # 마크다운 텍스트 업데이트 (이미지 참조 다음에 OCR 결과 삽입)
            original_match_string = match.group(0)
            replacement_string = f"{original_match_string}\n\n**OCR 텍스트:** {ocr_caption}\n"
            page_text = page_text[:match.start()] + replacement_string + page_text[match.end():]
        pages.append(Document(page_content=page_text))
        # final_markdown += page_text + "\n\n"

    save_pages(pages, output_path)
    return pages #final_markdown


def load_hwp5html_easyocr(hwp_filename, metadata = {}, max_chars_per_page=1500, min_items_per_page=2):
    """
    하이브리드 페이지 분리 규칙:
    - 글자 수 누적이 max_chars_per_page를 넘으면 페이지 분리
    - 제목(h1/h2/h3) 등장 시 페이지 분리
    - 이미지 앞뒤로 페이지 분리
    - 표도 글자 수에 포함
    """
    def hwp_to_html(hwp_path, temp_html_dir):
        """
        hwp5html을 사용해 HWP를 HTML로 변환
        temp_html_dir: HTML과 이미지가 저장될 폴더
        """
        os.makedirs(temp_html_dir, exist_ok=True)
        html_out_path = os.path.join(temp_html_dir, f"{firstname(hwp_path)}.html")  # 파일 경로 지정
        subprocess.run([ "hwp5html", "--output", html_out_path, "--html", hwp_path], check=True)
        with open(html_out_path, "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")    
        body = soup.body or soup
        return body
        
    def extract_table_md(table):
        rows = []
        for tr in table.find_all("tr"):
            cells = [c.get_text(strip=True) for c in tr.find_all(["td", "th"])]
            if cells:
                rows.append(cells)
        if not rows:
            return []
        md = []
        md.append("| " + " | ".join(rows[0]) + " |")
        md.append("| " + " | ".join(["---"] * len(rows[0])) + " |")
        for row in rows[1:]:
            md.append("| " + " | ".join(row) + " |")
        return md
    TEMP_HTML_DIR = "temp_hwp_html"
    hwp_path = f'files/{hwp_filename}'
    output_path = f'outputs/{ext(hwp_filename)}'
    if os.path.exists(output_path):
        return load_pages(output_path)
    
    # 1) HWP -> HTML    
    body = hwp_to_html(hwp_path, TEMP_HTML_DIR)

    # 2) HTML 파싱 + 하이브리드 페이지 분리 + EasyOCR
    reader = easyocr.Reader(['ko', 'en'], gpu=False)
    md_lines = []
    page_num = 1
    char_count = 0
    items_in_page = 0

    def new_page():
        nonlocal page_num, char_count, items_in_page
        if items_in_page >= min_items_per_page:
            md_lines.append("")  # 페이지 끝 공백 줄
            page_num += 1
            md_lines.append(f"## Page {page_num}")
            char_count = 0
            items_in_page = 0

    md_lines.append(f"## Page {page_num}")

    for elem in body.descendants:
        if isinstance(elem, Comment) or (isinstance(elem, NavigableString) and not str(elem).strip()):
            continue
        if not getattr(elem, "name", None):
            continue

        tag = elem.name.lower()

        # 제목 등장 시 페이지 분리
        if tag in ["h1", "h2", "h3"]:
            new_page()
            text = elem.get_text(" ", strip=True)
            if text:
                md_lines.append(text)
                char_count += len(text)
                items_in_page += 1
            continue

        # 문단
        if tag in ["p", "li"]:
            text = elem.get_text(" ", strip=True)
            if text and not elem.find(["table", "img"]):
                if char_count > max_chars_per_page:
                    new_page()
                md_lines.append(text)
                char_count += len(text)
                items_in_page += 1

        # 표
        if tag == "table":
            tbl_md = extract_table_md(elem)
            if tbl_md:
                if char_count > max_chars_per_page:
                    new_page()
                md_lines.extend(tbl_md)
                char_count += sum(len("".join(r)) for r in tbl_md)
                items_in_page += 1

        # 이미지
        if tag == "img":
            src = elem.get("src")
            if src:
                img_path = os.path.join(TEMP_HTML_DIR, os.path.basename(src))
                if os.path.exists(img_path):
                    # 이미지 앞에서 페이지 분리
                    if char_count > max_chars_per_page:
                        new_page()
                    ocr_text = reader.readtext(img_path, detail=0)
                    md_lines.append(f"![{os.path.basename(img_path)}]({img_path})")
                    md_lines.append("\n".join(ocr_text) if ocr_text else "_(인식된 텍스트 없음)_")
                    char_count += 50  # 이미지도 글자 수로 환산
                    items_in_page += 1
                    # 이미지 뒤에서 페이지 분리
                    new_page()

    pages = []
    pages_raw = re.split(r'## Page \d+\s*', "\n".join(md_lines).strip())
    for page_text in pages_raw:
        if page_text.strip():
            pages.append(Document(page_content=page_text.strip()))    
    save_pages(pages, output_path)
    return pages # "\n".join(md_lines).strip()


def load_documents(filename):
    def recursive_documents(documents):
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=150, separators=['\n\n', '\n', ' ', ''], length_function=len)
        documents = text_splitter.split_documents(documents)
        return documents
    
    _, ext = os.path.splitext(filename)
    if ext.lower() == '.pdf':
        documents = load_pymupdf4llm_easyocr(filename)
    elif ext.lower() == '.hwp':
        documents = load_hwp5html_easyocr(filename)
    else:
        print(f"'{filename}'는 지원하지 않는 파일 형식입니다.")
        
    return recursive_documents(documents)


### 필요자료 다운로드

In [7]:
download('1t9TWN25lsshk_tIXyh3Gx-NzfNAWRdeb', 'output.zip')

Downloading...
From (original): https://drive.google.com/uc?id=1t9TWN25lsshk_tIXyh3Gx-NzfNAWRdeb
From (redirected): https://drive.google.com/uc?id=1t9TWN25lsshk_tIXyh3Gx-NzfNAWRdeb&confirm=t&uuid=796e9da6-0fa8-4b60-ad22-531a0e41433c
To: /kaggle/working/output.zip
100%|██████████| 157M/157M [00:01<00:00, 91.8MB/s] 


[ok] output.zip 다운로드 완료.
[ok] output.zip 압축풀기 완료.


'output.zip'

## CSV 파일 확인하기

In [8]:
df = load_csv('data_list.csv')
df.head()

[ok] data_list.csv 로드 완료.


,공고 번호,공고 차수,사업명,사업 금액,발주 기관,공개 일자,입찰 참여 시작일,입찰 참여 마감일,사업 요약,파일형식,파일명,텍스트
0,20241001798,0.0,한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보시스템 고도화,130000000.0,한영대학,2024-10-04 13:51:23,NaN,2024-10-15 17:00:00,- 한영대학교 특성화 맞춤형 교육환경 구축을 위해 트랙운영 학사정보시스템을 고도화한...,hwp,한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp,\n \n2024년 특성화 맞춤형 교육환경 구축 – 트랙운영 학사정보시스템 ...
1,20241002912,0.0,2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선,129300000.0,한국연구재단,2024-10-04 15:01:52,2024-10-14 10:00:00,2024-10-16 14:00:00,- 사업 개요: 2024년 대학 산학협력활동 실태조사 시스템(UICC) 기능개선\n...,hwp,한국연구재단_2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선.hwp,\r\n \r\n \r\n \r\n제 안 요 청 서\r\n[ 2024년 대학 ...
2,20240827859,0.0,EIP3.0 고압가스 안전관리 시스템 구축 용역,40000000.0,한국생산기술연구원,2024-08-28 11:31:02,2024-08-29 09:00:00,2024-09-09 10:00:00,- 사업 개요: EIP3.0 고압가스 안전관리 시스템 구축 용역\n- 추진배경: 안...,hwp,한국생산기술연구원_EIP3.0 고압가스 안전관리 시스템 구축 용역.hwp,\r\n \r\nEIP3.0 고압가스 안전관리\r\n시스템 구축 용역\...
3,20240430918,0.0,도시계획위원회 통합관리시스템 구축용역,150000000.0,인천광역시,2024-04-18 16:26:32,2024-05-02 10:00:00,2024-05-09 16:00:00,- 사업명: 도시계획위원회 통합관리시스템 구축 용역\n- 용역개요: 도시계획위원회와...,hwp,인천광역시_도시계획위원회 통합관리시스템 구축용역.hwp,\r\n \r\n \r\n도시계획위원회 통합관리시스템 구축\r\n제 안 요 청...
4,20240430896,0.0,봉화군 재난통합관리시스템 고도화 사업(협상)(긴급),900000000.0,경상북도 봉화군,2024-04-18 16:33:28,2024-04-26 09:00:00,2024-04-30 17:00:00,- 사업명: 봉화군 재난통합관리시스템 고도화 사업\n- 사업개요: 공동수급(공동이행...,hwp,경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).hwp,\r\n \r\n \r\n제안요청서\r\n \r\n사 업 명\r\n봉화...


## PDF 샘플 로딩

In [8]:
pdf_df = df[df['파일형식'] == 'pdf']
pdf_filename = pdf_df.iloc[0]['파일명'] # 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf
pages = load_pymupdf4llm_easyocr(pdf_filename) # 추후 pages를 반환하도록 변경해야 함.
markdown_output = '\n\n---\n\n'.join([p.page_content for p in pages])
with open(ext(pdf_filename, 'md'), "w", encoding="utf-8") as f:
    f.write(markdown_output)
print(f"PDF 처리 완료. {ext(pdf_filename, 'md')} 파일 생성.")

Progress: |██████████████████████████████████████████████████| 100.1% Complete페이지:  1
페이지:  2
페이지:  3
페이지:  4
페이지:  5
페이지:  6
페이지:  7
페이지:  8
페이지:  9
페이지:  10
페이지:  11
페이지:  12
페이지:  13
페이지:  14
페이지:  15
페이지:  16
페이지:  17
페이지:  18
페이지:  19
페이지:  20
페이지:  21
페이지:  22
페이지:  23
페이지:  24
페이지:  25
페이지:  26
페이지:  27
페이지:  28
페이지:  29
페이지:  30
페이지:  31
페이지:  32
페이지:  33
페이지:  34
페이지:  35
페이지:  36
페이지:  37
페이지:  38
페이지:  39
페이지:  40
페이지:  41
페이지:  42
페이지:  43
페이지:  44
페이지:  45
페이지:  46
페이지:  47
페이지:  48
페이지:  49
페이지:  50
페이지:  51
페이지:  52
페이지:  53
페이지:  54
페이지:  55
페이지:  56
페이지:  57
페이지:  58
페이지:  59
페이지:  60
페이지:  61
페이지:  62
페이지:  63
페이지:  64
페이지:  65
페이지:  66
페이지:  67
페이지:  68
페이지:  69
페이지:  70
페이지:  71
페이지:  72
페이지:  73
페이지:  74
페이지:  75
페이지:  76
페이지:  77
페이지:  78
페이지:  79
페이지:  80
페이지:  81
페이지:  82
페이지:  83
페이지:  84
페이지:  85
페이지:  86
페이지:  87
페이지:  88
페이지:  89
페이지:  90
페이지:  91
페이지:  92
페이지:  93
페이지:  94
페이지:  95
페이지:  96
페이지:  97
페이지:  98
페이지:  99
페이지:  100
페이지:  101
페이지:  102
페이지:  103


## HWP 샘플 로딩

In [9]:
# 예시 사용
hwp_filename = df.iloc[0]['파일명'] # 한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp
print(f"원본 파일명: {hwp_filename}")
print(f".pkl 파일명: {ext(hwp_filename)}")

filename_to_find = '전북대학교_JST 공유대학(원) xAPI기반 LRS시스템 구축.hwp'
found_row = df[df['파일명'] == filename_to_find]

if not found_row.empty:
    print(f"'{filename_to_find}' 파일을 찾았습니다.")
    display(found_row)
else:
    print(f"'{filename_to_find}' 파일을 찾을 수 없습니다.")


원본 파일명: 한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp
.pkl 파일명: 한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.pkl
'전북대학교_JST 공유대학(원) xAPI기반 LRS시스템 구축.hwp' 파일을 찾았습니다.


,공고 번호,공고 차수,사업명,사업 금액,발주 기관,공개 일자,입찰 참여 시작일,입찰 참여 마감일,사업 요약,파일형식,파일명,텍스트
20,20240903688,0.0,JST 공유대학(원) xAPI기반 LRS시스템 구축,116000000.0,전북대학교,2024-09-04 15:01:14,2024-09-05 09:00:00,2024-09-19 12:00:00,- 사업개요: JST 공유대학(원)에서 xAPI 기반 LRS시스템을 구축하는 사업\...,hwp,전북대학교_JST 공유대학(원) xAPI기반 LRS시스템 구축.hwp,\r\n \r\n \r\n \r\n JST 공유대학(원) xAPI기반 LR...


In [13]:
# ===== 실행 예시 =====
hwp_filename = "전북대학교_JST 공유대학(원) xAPI기반 LRS시스템 구축.hwp"
documents = load_hwp5html_easyocr(hwp_filename)
markdown_output = '\n\n---\n\n'.join([p.page_content for p in documents])
with open(ext(hwp_filename, 'md'), "w", encoding="utf-8") as f:
    f.write(markdown_output)
print(f"PDF 처리 완료. {ext(hwp_filename, 'md')} 파일 생성.")
print(markdown_output[:1200])

INFO     hwp5.plat.javax_transform: disabled
INFO     compile typedef of <class 'hwp5.filestructure.FileHeader'>
INFO     compile typedef of <class 'hwp5.msoleprops.PropertySetStreamHeader'>
INFO     compile typedef of <class 'hwp5.msoleprops.PropertySetHeader'>
INFO     compile typedef of <class 'hwp5.msoleprops.Dictionary'>
INFO     compile typedef of <class 'hwp5.msoleprops.TypedPropertyValue'>
INFO     compile typedef of <class 'hwp5.dataio.UINT32'>
INFO     compile typedef of <class 'hwp5.dataio.INT32'>
INFO     filter compiled typedef of <class 'hwp5.binmodel.tagid16_document_properties.DocumentProperties'> with version (5, 1, 0, 1)
INFO     compile typedef of <class 'hwp5.binmodel.tagid16_document_properties.DocumentProperties'>
INFO     filter compiled typedef of <class 'hwp5.binmodel.tagid17_id_mappings.IdMappings'> with version (5, 1, 0, 1)
INFO     compile typedef of <class 'hwp5.binmodel.tagid17_id_mappings.IdMappings'>
INFO     filter compiled typedef of <class 'hwp5.binmo

NameError: name 'out_dir' is not defined

## 백터스토어 생성

In [29]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda


def create_context():
    VERSION = 'v1'
    EMB_NAME = 'intfloat/multilingual-e5-large-instruct'
    STORED_FOLDER = f'faiss-{VERSION}'
    os.environ["GOOGLE_API_KEY"] = load_secret("GOOGLE_API_KEY")
    os.environ["OPENAI_API_KEY"] = load_secret("OPENAI_API_KEY")
    embedding = HuggingFaceEmbeddings(model_name=EMB_NAME, multi_process=True) # GPU가 2개이기 때문임
    

    def save_vectorstore(documents):
        # 새로운 문서가 있을 경우, 기존 인덱스에 추가
        print(f"{len(documents)}개의 새로운 문서를 vectorstore에 추가합니다.")
        vectorstore.add_documents(documents=documents)        
        vectorstore.save_local(STORED_FOLDER) # 변경된 인덱스 저장
        print(f"업데이트된 vectorstore를 '{STORED_FOLDER}' 폴더에 저장했습니다.")
        return vectorstore

    def print_pipe(x):
        print("--- 랭체인 중간 결과 ---")
        print(x)
        print("-------------------------")
        return x
    
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)
    
    def load_llm_gemini():
        llm = ChatGoogleGenerativeAI(
            model="gemini-2.5-flash",
            temperature=0.2,
            thinking_budget=0,  # reasoning 비활성화
            max_output_tokens=1000,  # HuggingFacePipeline의 max_new_tokens에 해당
        )
        return llm

    def create_chain(retriever, llm):
        prompt = PromptTemplate(
            template=(
                "다음 문맥만을 근거로 질문에 답변하세요.\n"
                "- 반드시 한국어로 답변하세요.\n"
                "- 문맥에 없는 내용은 '모르겠습니다'라고 답하세요.\n"
                "- 금액/기한/요건은 원문과 일치하게 유지하세요.\n\n"
                "문맥:\n{context}\n\n"
                "질문:\n{question}\n\n"
                "답변:"
            ),
            input_variables=["context", "question"]
        )
        chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt | llm | StrOutputParser()
        )
        return chain

    def create_chain_gemini(retriever, llm):
        prompt = ChatPromptTemplate.from_messages([
            ("system",
             "다음 문맥만을 근거로 질문에 답변하세요.\n"
             "- 반드시 한국어로 답변하세요.\n"
             "- 문맥에 없는 내용은 '모르겠습니다'라고 답하세요.\n"
             "- 금액/기한/요건은 원문과 일치하게 유지하세요."
            ),
            ("human", "문맥:\n{context}\n\n질문:\n{question}")
        ])
    
        chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt | RunnableLambda(print_pipe) | llm | RunnableLambda(print_pipe) | StrOutputParser()
        )
        return chain
    
    # 1. 기존 FAISS 인덱스 로드 또는 새로 생성
    if os.path.exists(STORED_FOLDER):
        vectorstore = FAISS.load_local(STORED_FOLDER, embedding, allow_dangerous_deserialization=True)
        print(f"'{STORED_FOLDER}' 폴더에서 vectorstore를 로드했습니다.")
    else:
        # 인덱스가 없을 경우, 더미 문서를 사용해 인덱스 초기 생성
        dummy_docs = [Document(page_content="초기 생성을 위한 더미 문서입니다.")]
        vectorstore = FAISS.from_documents(documents=dummy_docs, embedding=embedding)
        print(f"새로운 vectorstore를 더미 문서로 생성했습니다.")
    
    retriever = vectorstore.as_retriever()
    llm = load_llm_gemini()
    chain = create_chain_gemini(retriever, llm)
    return save_vectorstore, chain
    
hwp_filename = "전북대학교_JST 공유대학(원) xAPI기반 LRS시스템 구축.hwp"
documents = load_documents(hwp_filename)
save_vectorstore, chain = create_context()
vectorstore = save_vectorstore(documents)

'outputs/전북대학교_JST 공유대학(원) xAPI기반 LRS시스템 구축.pkl' 파일에서 pages를 로드했습니다.
'faiss-v1' 폴더에서 vectorstore를 로드했습니다.
90개의 새로운 문서를 vectorstore에 추가합니다.


2025-09-15 09:11:47.682208: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757927507.704478    1555 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757927507.711239    1555 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-15 09:11:56.465456: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757927516.488349    1567 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757927516.495244    1567 cuda_blas.cc:1

Chunks:   0%|          | 0/18 [00:00<?, ?it/s]

업데이트된 vectorstore를 'faiss-v1' 폴더에 저장했습니다.


In [30]:
print(timer(lambda: chain.invoke("컨설팅 요구사항에 대해서 설명해 주세요.")))

2025-09-15 09:12:11.202859: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757927531.224784    1596 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757927531.231507    1596 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-15 09:12:19.978629: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757927540.000357    1608 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757927540.007109    1608 cuda_blas.cc:1

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

--- 랭체인 중간 결과 ---
messages=[SystemMessage(content="다음 문맥만을 근거로 질문에 답변하세요.\n- 반드시 한국어로 답변하세요.\n- 문맥에 없는 내용은 '모르겠습니다'라고 답하세요.\n- 금액/기한/요건은 원문과 일치하게 유지하세요.", additional_kwargs={}, response_metadata={}), HumanMessage(content='문맥:\n구   분\n설        명\n개수\n컨설팅- CNR\n(Consulting Requirement)\n업무 효율성과 생산성을 높이기 위한 정보시스템 구축, 업무 프로세스 개선 방안 등의 도출을 위한 요구사항\n2\n시스템 장비구성- ECR\n(Equipment Composition\nRequirement)\n목표사업수행을 위해 필요한 하드웨어, 소프트웨어, 네트워크 등의 도입 장비 내역 등 시스템 장비 구성에 대한 요구사항을 기술\n4\n기능- SFR\n(System Function\nRequirement)\n목표시스템이 반드시 수행하여야 하거나 목표시스템을 이용하여 사용 자가 반드시 할 수 있어야 하는 기능에 대한 기술\n단, 개별 기능요구사항은 전체 시스템의 계층적 구조분석을 통해 단 위업무별 기능구조를 도출한 후, 이에 대한 세부 기능별 상세 요구사 항을 작성하는 것을 원칙으로 하며, 기능 수행을 위한 데이터 요구사 항과 연계를 고려하여 기술함\n6\n성능- PER\n(Performance Requirement)\n목표시스템의 처리속도 및 시간, 처리량, 동적 ․ 정적용량, 가용성 등 성 능에 대한 요구사항을 기술\n2\n인터페이스- SIR\n(System Interface\nRequirement)\n목표시스템과 외부를 연결하는 시스템인터페이스와 사용자인터페이스 에 대한 요구사항을 기술\n1\n데이터- DAR\n(Data Requirement)\n목표 시스템의 서비스에 필요한 초기자료 구축 및 데이터 변환을 위한 대상, 방법, 보안이 필요한 데이터 등 